<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_MeanShiftVsTFIDF_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#SUBJECT: Mean-shift Vs TF-IDF

###AUTHOR: Sheida Talei

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.cluster import MeanShift , estimate_bandwidth
from sklearn.decomposition import TruncatedSVD

In [11]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [12]:

X_and_Y = removeEmptyRows ('/content/drive/MyDrive/Final/train_temp.csv')
Y= X_and_Y.Label
X = X_and_Y.text

In [13]:
#--------------------------------------------------Loading StopWords ------------------------------------------
#Source of file: https://sites.google.com/site/kevinbouge/stopwords-lists
def getStopWord ():
    try:
        file = open('/content/drive/MyDrive/Final/stopwords_fa.txt', 'r', encoding='utf-8-sig')
        file_readed = file.read()
    
    finally:
        file.close()
        
    stopWord_Set = set(file_readed.split())
    return stopWord_Set

In [14]:
persian_stop_word = list(getStopWord())

In [15]:
vectorizer = TfidfVectorizer(stop_words= persian_stop_word ,  max_features = 2000 ) 
X_train_vector  = vectorizer.fit_transform(X)

In [16]:
#source: https://stackoverflow.com/questions/34725726/is-it-possible-apply-pca-on-any-text-classification
svd = TruncatedSVD(n_components=200)
X_svd = svd.fit_transform(X_train_vector)

In [17]:
#source: https://pythonprogramming.net/hierarchical-clustering-mean-shift-machine-learning-tutorial/
#source: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html
#source: https://scikit-learn.org/stable/auto_examples/cluster/plot_mean_shift.html
#If the chosen value is too big, then the clusters will tend to combine and the final output will be a smaller number of clusters than desired.
# If the chosen value is too small, then the algorithm may produce too many clusters and it will take longer to run.

#source:https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/modules/generated/sklearn.cluster.estimate_bandwidth.html#sklearn.cluster.estimate_bandwidth
# bandwidth = estimate_bandwidth(X_train_vector.toarray(), quantile=0.2, n_samples=500)

clustering = MeanShift()
clustering.fit(X_svd)


MeanShift()

In [18]:
labels = clustering.labels_
silhouette_score = metrics.silhouette_score(X_svd, labels, metric='euclidean')
print ("Silhouette_score: ",silhouette_score )

Silhouette_score:  0.2329766821465387


In [ ]:
labels = clustering.labels_

In [19]:
cluster_centers = clustering.cluster_centers_
print(cluster_centers)

n_clusters_ = len(np.unique(labels))
print("Number of estimated clusters:", n_clusters_)

[[ 1.00333170e-01 -7.25785044e-03 -1.35548383e-02 ...  1.22845164e-04
   4.40775048e-04 -8.13436151e-04]
 [ 1.98905879e-01 -5.91866018e-01  7.06882866e-01 ...  1.64381933e-03
  -4.47896326e-04 -2.53790521e-03]
 [ 1.57777607e-01 -1.29664684e-01 -1.24629711e-01 ... -4.40957356e-03
   9.11103327e-03  9.11334543e-03]
 ...
 [ 2.31452596e-01 -1.22347239e-01 -6.88847221e-02 ...  2.06543181e-02
   1.50130055e-02 -4.83537334e-02]
 [ 1.87093464e-01  2.16132758e-02 -7.10440670e-02 ... -4.37770365e-04
  -1.06578074e-02 -5.63705324e-03]
 [ 1.66176610e-01  1.36658462e-02 -7.05808728e-02 ... -2.79859033e-03
  -1.94851950e-02  7.37642597e-03]]
Number of estimated clusters: 8


In [21]:
# clustering.predict(X_train_vector)